In [1]:
import folium
import pandas as pd

processed_cdr_df = pd.read_pickle("datasets/processed_cdr.pkl")
# Sample the processed data
sample_df = processed_cdr_df.sample(30000)#.to_crs(epsg=4326)  # adjust N

# Create a folium map centered on the data
m = folium.Map(location=[38.7169, -9.1399], zoom_start=13)

# Add estimated points
for _, row in sample_df.iterrows():
    folium.CircleMarker(
        location=[row["est_lat"], row["est_lon"]],
        radius=2,
        color="blue",
        fill=True,
        fill_opacity=0.5,
        popup=f"user: {row['unique_id']}, cell: {row['cell_id']}"
    ).add_to(m)

# Optionally add tower locations
towers = sample_df[["longitude_cell", "latitude_cell", "cell_id"]].drop_duplicates()
for _, row in towers.iterrows():
    folium.Marker(
        location=[row["latitude_cell"], row["longitude_cell"]],
        icon=folium.Icon(color="red", icon="signal"),
        popup=f"Tower: {row['cell_id']}"
    ).add_to(m)

m.save("spreaded_points.html")
#m


In [2]:
processed_cdr_df

,unique_id,time_id,event_date,a_bts_cgi,longitude_cell,latitude_cell,cgi_key,cell_id,r,azi_min1,azi_max1,concelho,new_radius,est_lon,est_lat
0,08GcaR5hWj9xkKRuUeE07UHAv+zEUjOq/i1VZWPtcPPM=_...,2024-02-09 08:58:26.0,2024-02-09,2664202510,-9.038759,38.897488,2.664203e+09,2664202510,154.0,75.0,200.0,VILA FRANCA DE XIRA,531.65,-9.036183,38.898421
1,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-09 11:56:21.0,2024-02-09,288607000027,-9.230803,38.702776,2.886070e+11,288607000027,127.0,30.0,150.0,OEIRAS,127.00,-9.230261,38.702016
2,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-27 20:13:20.0,2024-02-27,297281400028,-9.147932,38.707327,2.972814e+11,297281400028,64.0,0.0,0.0,LISBOA,64.00,-9.147642,38.707685
3,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-10 18:15:28.0,2024-02-10,291295000015,-9.263709,38.759208,2.912950e+11,291295000015,158.0,120.0,240.0,SINTRA,158.00,-9.264624,38.760131
4,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-02-05 16:29:36.0,2024-02-05,293496600002,-9.112000,38.757891,2.934966e+11,293496600002,241.0,90.0,270.0,LISBOA,241.00,-9.112817,38.759189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43748229,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-07 08:23:50,2024-03-07,287096700015,-9.268195,38.757820,2.870967e+11,287096700015,210.0,-100.0,185.0,SINTRA,210.00,-9.269123,38.756538
43748230,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-09 14:24:57,2024-03-09,292446100027,-9.238561,38.725664,2.924461e+11,292446100027,47.0,290.0,410.0,OEIRAS,47.00,-9.238570,38.725274
43748231,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-07 11:58:33,2024-03-07,281873100001,-9.107435,38.814361,2.818731e+11,281873100001,314.0,340.0,100.0,LOURES,314.00,-9.105541,38.813945
43748232,0YB4n38/GIE68JTW8IPie+j3peE8LjaMCaNbOl466lXg=_...,2024-03-05 17:47:06,2024-03-05,285868900002,-9.138016,38.742714,2.858689e+11,285868900002,139.0,280.0,400.0,LISBOA,139.00,-9.138548,38.742718


In [19]:
import folium
import geopandas as gpd
# Reproject sectors to WGS84 for Folium
sectors_gdf = gpd.read_file("sectors.geojson")
sectors_web = sectors_gdf.to_crs(epsg=4326)

single_sector = sectors_web[sectors_web["cell_id"] == 711306010]

# Center the map on Lisbon
lisbon_center = [38.7169, -9.1399]
m = folium.Map(location=lisbon_center, zoom_start=12)

# Add the sector polygons to the map
# Add the sector polygons with cell_id tooltip
folium.GeoJson(
    single_sector,
    tooltip=folium.GeoJsonTooltip(fields=["cell_id"], aliases=["Cell ID:"])
).add_to(m)

# Show in notebook (if using Jupyter)
m

# Save to HTML for viewing in browser
#m.save("sectors_map.html")


In [6]:
single_sector

,cell_id,geometry
